In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
df=pd.read_csv('cardekho_imputated.csv',index_col=[0])

In [3]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [4]:
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [5]:
df.drop('car_name',axis=1,inplace=True)
df.drop('brand',axis=1,inplace=True)

In [6]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [7]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [8]:
num_features=[feature for feature in df.columns if df[feature].dtype != 'O']
print("Numerical Features:", num_features)
cat_features=[feature for feature in df.columns if df[feature].dtype == 'O']
print("Categorical Features:", cat_features)
discrete_features = [feature for feature in num_features if len(df[feature].unique()) <= 25]
print("Discrete Features:", discrete_features)
continuous_features = [feature for feature in num_features if feature not in discrete_features]
print("Continuous Features:", continuous_features)

Numerical Features: ['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats', 'selling_price']
Categorical Features: ['model', 'seller_type', 'fuel_type', 'transmission_type']
Discrete Features: ['vehicle_age', 'seats']
Continuous Features: ['km_driven', 'mileage', 'engine', 'max_power', 'selling_price']


In [9]:
from sklearn.model_selection import train_test_split
X=df.drop(['selling_price'],axis=1)
y=df['selling_price']

In [10]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [11]:
len(df['model'].unique())

120

In [12]:
df['model'].value_counts()

model
i20             906
Swift Dzire     890
Swift           781
Alto            778
City            757
               ... 
Altroz            1
C                 1
Ghost             1
Quattroporte      1
Gurkha            1
Name: count, Length: 120, dtype: int64

In [13]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['model']=le.fit_transform(X['model'])

In [14]:
num_features=X.select_dtypes(exclude='object').columns
onehot_columns=['seller_type','fuel_type','transmission_type']
label_encoded_columns=['model']

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder(drop='first')
preprocessor=ColumnTransformer(
    [
        ('OneHotEncoder',oh_transformer,onehot_columns),
        ('StandardScaler',numeric_transformer,num_features),
    ], remainder='passthrough'
)


In [15]:
X=preprocessor.fit_transform(X)

In [16]:
X

array([[ 1.        ,  0.        ,  0.        , ..., -1.32425883,
        -1.26335238, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.55471774,
        -0.43257082, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.55471774,
        -0.47911321, -0.40302241],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  0.02291783,
         0.06822523, -0.40302241],
       [ 0.        ,  0.        ,  1.        , ...,  1.32979434,
         0.91715831,  2.07344426],
       [ 0.        ,  0.        ,  0.        , ...,  0.02099878,
         0.39588361, -0.40302241]])

In [17]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [18]:
X_train

array([[ 0.        ,  0.        ,  1.        , ...,  1.75390551,
         2.66249771, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.55087963,
        -0.38602844, -0.40302241],
       [ 0.        ,  0.        ,  1.        , ...,  0.89033072,
         3.27453006, -0.40302241],
       ...,
       [ 1.        ,  0.        ,  0.        , ..., -0.9366097 ,
        -0.78070786, -0.40302241],
       [ 0.        ,  0.        ,  0.        , ..., -0.55471774,
        -0.43582879, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.04616815,
         0.06194201, -0.40302241]])

In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [27]:
def evaluate_model(true,predicted):
    r2=r2_score(true,predicted)
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mse)
    
    print(f"R2 Score: {r2:.4f}")
    print(f"Mean Absolute Error: {mae:.4f}")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"Root Mean Squared Error: {rmse:.4f}")

In [28]:
models={
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor()
}
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    print(f"Model: {list(models.keys())[i]}")
    print("Training Set Evaluation:")
    evaluate_model(y_train,y_train_pred)

Model: LinearRegression
Training Set Evaluation:
R2 Score: 0.6218
Mean Absolute Error: 268101.6071
Mean Squared Error: 306756099359.7596
Root Mean Squared Error: 553855.6665
Model: Ridge
Training Set Evaluation:
R2 Score: 0.6218
Mean Absolute Error: 268059.8015
Mean Squared Error: 306756818740.9266
Root Mean Squared Error: 553856.3160
Model: Lasso
Training Set Evaluation:
R2 Score: 0.6218
Mean Absolute Error: 268099.2226
Mean Squared Error: 306756104248.3742
Root Mean Squared Error: 553855.6710
Model: KNeighborsRegressor
Training Set Evaluation:
R2 Score: 0.8691
Mean Absolute Error: 91425.4705
Mean Squared Error: 106193217845.5548
Root Mean Squared Error: 325873.0088
Model: DecisionTreeRegressor
Training Set Evaluation:
R2 Score: 0.9995
Mean Absolute Error: 5164.8199
Mean Squared Error: 432524990.5364
Root Mean Squared Error: 20797.2352
Model: RandomForestRegressor
Training Set Evaluation:
R2 Score: 0.9803
Mean Absolute Error: 39751.2649
Mean Squared Error: 15940258799.3429
Root Mean S

In [29]:
knn_params={"n_neighbors": [2, 3, 10, 20, 40, 50]}
rf_params={"max_depth": [5,8,15,None,10],
           "max_features": [5,7,"auto",8],
           "min_samples_split": [2,8,15,20],
           "n_estimators": [100,200,500,1000]}
ada_params={"n_estimators": [50, 60, 70,80, 90],
            "loss":['linear', 'square', 'exponential']}

In [31]:
randomcv_models=[('KNN', KNeighborsRegressor(),knn_params),
                 ('RF',RandomForestRegressor(),rf_params),
                 ('AdaBoost', AdaBoostRegressor(),ada_params)]

In [32]:
from sklearn.model_selection import RandomizedSearchCV
model_params={}
for name,model,params in randomcv_models:
    random=RandomizedSearchCV(estimator=model,
                              param_distributions=params,
                             n_iter=10,
                             cv=3,
                             verbose=2,
                             n_jobs=-1)
    random.fit(X_train,y_train)
    model_params[name]=random.best_params_

    for model_name in model_params:
        print(f"---------------Best Parameters for {model_name}---------------")
        print(model_params[model_name])

c:\Users\Taraksh Goyal\Desktop\coding\myenv\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 6 candidates, totalling 18 fits
---------------Best Parameters for KNN---------------
{'n_neighbors': 10}
Fitting 3 folds for each of 10 candidates, totalling 30 fits


c:\Users\Taraksh Goyal\Desktop\coding\myenv\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
9 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
7 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Taraksh Goyal\Desktop\coding\myenv\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Taraksh Goyal\Desktop\coding\myenv\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\Taraksh Goyal\Desktop\coding\myenv\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_

---------------Best Parameters for KNN---------------
{'n_neighbors': 10}
---------------Best Parameters for RF---------------
{'n_estimators': 200, 'min_samples_split': 2, 'max_features': 7, 'max_depth': 15}
Fitting 3 folds for each of 10 candidates, totalling 30 fits
---------------Best Parameters for KNN---------------
{'n_neighbors': 10}
---------------Best Parameters for RF---------------
{'n_estimators': 200, 'min_samples_split': 2, 'max_features': 7, 'max_depth': 15}
---------------Best Parameters for AdaBoost---------------
{'n_estimators': 50, 'loss': 'linear'}


In [33]:
models={
    "RandomForestRegressor": RandomForestRegressor(**model_params['RF']),
    "KNeighborsRegressor": KNeighborsRegressor(**model_params['KNN']),
    "AdaBoostRegressor": AdaBoostRegressor(**model_params['AdaBoost'])
}
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    print(f"Model: {list(models.keys())[i]}")
    print("Training Set Evaluation:")
    evaluate_model(y_train,y_train_pred)
    print("Test Set Evaluation:")
    evaluate_model(y_test,y_test_pred)
    print("\n")

Model: RandomForestRegressor
Training Set Evaluation:
R2 Score: 0.9783
Mean Absolute Error: 54413.1098
Mean Squared Error: 17631788960.9576
Root Mean Squared Error: 132784.7467
Test Set Evaluation:
R2 Score: 0.9372
Mean Absolute Error: 98251.6651
Mean Squared Error: 47272749152.4088
Root Mean Squared Error: 217422.9729


Model: KNeighborsRegressor
Training Set Evaluation:
R2 Score: 0.8371
Mean Absolute Error: 103472.0474
Mean Squared Error: 132103731766.7099
Root Mean Squared Error: 363460.7706
Test Set Evaluation:
R2 Score: 0.9075
Mean Absolute Error: 117496.2131
Mean Squared Error: 69636909424.4648
Root Mean Squared Error: 263888.0623


Model: AdaBoostRegressor
Training Set Evaluation:
R2 Score: 0.7174
Mean Absolute Error: 374841.5046
Mean Squared Error: 229226374706.3872
Root Mean Squared Error: 478775.9128
Test Set Evaluation:
R2 Score: 0.6606
Mean Absolute Error: 389536.4904
Mean Squared Error: 255469051441.7779
Root Mean Squared Error: 505439.4637


